In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pwd

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
import keras
import os
from PIL import Image
import natsort
import io
import cv2

In [ ]:
cd drive/MyDrive/[한이음] 적대적 AI 공격에 대한 인공지능 보안기술 연구/3. 소스코드/Attacked_GTSRB

In [ ]:
# 43개 클래스 모두 사용
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [ ]:
import natsort

imgs_path = "Attacked_train"
data_list = []
labels_list = []

for img in natsort.natsorted(os.listdir(imgs_path)):
    im = Image.open(imgs_path +'/'+ img)
    im = im.resize((32,32))
    im = np.array(im)
    data_list.append(im)
data_attacked_train = np.array(data_list)

labels_list_attacked_train = pd.read_csv('predictions_train.csv')

labels_test_attacked_train = np.array(labels_list_attacked_train['Original'])
print("Done")

In [ ]:
data_attacked_train.shape

In [ ]:
len(labels_test_attacked_train)

In [ ]:
import natsort

imgs_path = "Attacked_test"
data_list = []
labels_list = []

for img in natsort.natsorted(os.listdir(imgs_path)):
    im = Image.open(imgs_path +'/'+ img)
    im = im.resize((32,32))
    im = np.array(im)
    data_list.append(im)
data_attacked_test = np.array(data_list)

labels_list_attacked_test = pd.read_csv('predictions_test.csv')

labels_test_attacked_test = np.array(labels_list_attacked_test['Original'])
print("Done")

In [ ]:
data_attacked_test.shape

In [ ]:
len(labels_test_attacked_test)

In [ ]:
import natsort

imgs_path = "Original_train"
data_list = []
labels_list = []

for img in natsort.natsorted(os.listdir(imgs_path)):
    im = Image.open(imgs_path +'/'+ img)
    im = im.resize((32,32))
    im = np.array(im)
    data_list.append(im)
data_original_train = np.array(data_list)

labels_list_original_train = pd.read_csv('predictions_train.csv')

labels_test_original_train = np.array(labels_list_original_train['Original'])
print("Done")

In [ ]:
data_original_train.shape

In [ ]:
import natsort

imgs_path = "Original_test"
data_list = []
labels_list = []

for img in natsort.natsorted(os.listdir(imgs_path)):
    im = Image.open(imgs_path +'/'+ img)
    im = im.resize((32,32))
    im = np.array(im)
    data_list.append(im)
data_original_test = np.array(data_list)

labels_list_original_test = pd.read_csv('predictions_test.csv')

labels_test_original_test = np.array(labels_list_original_test['Original'])
print("Done")

In [ ]:
data_original_test.shape

In [ ]:
for i in range(70,80,1):
    print('attacked', labels_test_attacked_train[i])
    plt.imshow(data_attacked_train[i])
    plt.show()
    print('original', labels_test_original_train[i])
    plt.imshow(data_original_train[i])
    plt.show()
    print('=========================================')

### Adversarial Training

적대적 학습을 위해서는 공격 받은 데이터와 원본데이터를 섞어야한다.

가장 기본적으로 

1. train의 원본, 공격 데이터를 50%씩 뽑아서 학습하고 test set으로 성능을 확인해보겠다.

In [ ]:
total_list = []
for i in range(len(data_attacked_train)):
    total_list.append(i)

print(total_list)
print(len(total_list))

In [ ]:
import random

attack_list = []
ran_num = random.randint(0,len(data_attacked_train))

for i in range(int(len(data_attacked_train)/2)):
    while ran_num in attack_list:
        ran_num = random.randint(0,len(data_attacked_train) - 1)
    attack_list.append(ran_num)

attack_list.sort()
print(attack_list)
print(len(attack_list))

In [ ]:
original_list = list(set(total_list) - set(attack_list))

print('original list : ',original_list)
print(len(original_list))
print('attack list : ',attack_list)
print(len(attack_list))

In [ ]:
# attack (50%) + original (50%)로 데이터 셋을 구성
train_data = []
for i in total_list:
    if i in attack_list:
        train_data.append(data_attacked_train[i])
    else:
        train_data.append(data_original_train[i])
train_data = np.array(train_data)

print(train_data.shape)

In [ ]:
for i in range(10):
    plt.imshow(train_data[i])
    plt.show()

In [ ]:
train_R = train_data[:,:,:,0]
train_G = train_data[:,:,:,1]
train_B = train_data[:,:,:,2]

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

class ANN:
    def __init__(self, layers_size):
        self.layers_size = layers_size
        self.parameters = {}
        self.L = len(self.layers_size)
        self.n = 0
        self.costs = []
    
    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def softmax(self, Z):
        expZ = np.exp(Z - np.max(Z))
        return expZ / expZ.sum(axis=0, keepdims=True)
    
    def initialize_parameters(self):
        for l in range(1, len(self.layers_size)):
            self.parameters["W" + str(l)] = np.random.randn(self.layers_size[l], self.layers_size[l - 1]) / np.sqrt(self.layers_size[l - 1])
            self.parameters["b" + str(l)] = np.zeros((self.layers_size[l], 1))
    
    def forward(self, X):
        store = {}
 
        A = X.T
        for l in range(self.L - 1):
            Z = self.parameters["W" + str(l + 1)].dot(A) + self.parameters["b" + str(l + 1)]
            A = self.sigmoid(Z)
            store["A" + str(l + 1)] = A
            store["W" + str(l + 1)] = self.parameters["W" + str(l + 1)]
            store["Z" + str(l + 1)] = Z
 
        Z = self.parameters["W" + str(self.L)].dot(A) + self.parameters["b" + str(self.L)]
        A = self.softmax(Z)
        store["A" + str(self.L)] = A
        store["W" + str(self.L)] = self.parameters["W" + str(self.L)]
        store["Z" + str(self.L)] = Z
 
        return A, store
 
    def sigmoid_derivative(self, Z):
        s = 1 / (1 + np.exp(-Z))
        return s * (1 - s)

    def backward(self,X,Y,store):

        derivatives = {}

        store['A0'] = X.T

        A = store['A' + str(self.L)]
        dZ = A - Y.T

        dW = dZ.dot(store['A' + str(self.L - 1)].T) / self.n
        db = np.sum(dZ, axis = 1, keepdims = True) / self.n
        dAPrev = store['W' + str(self.L)].T.dot(dZ)

        derivatives['dW' + str(self.L)] = dW
        derivatives['db' + str(self.L)] = db

        for l in range(self.L - 1,0,-1):
            dZ = dAPrev * self.sigmoid_derivative(store['Z' + str(l)])
            dW = dZ.dot(store['A' + str(l-1)].T) / self.n
            db = np.sum(dZ, axis = 1, keepdims = True) / self.n
            if l > 1:
                dAPrev = store['W' + str(l)].T.dot(dZ)
            derivatives["dW" + str(l)] = dW
            derivatives["db" + str(l)] = db

        return derivatives
    
    def backward_ad(self,X,Y,store):

        derivatives = {}

        store['A0'] = X.T

        A = store['A' + str(self.L)]
        dZ = A - Y.T

        dW = dZ.dot(store['A' + str(self.L - 1)].T) / self.n
        db = np.sum(dZ, axis = 1, keepdims = True) / self.n
        dAPrev = store['W' + str(self.L)].T.dot(dZ)

        derivatives['dW' + str(self.L)] = dW
        derivatives['db' + str(self.L)] = db

        for l in range(self.L - 1,0,-1):
            dZ = dAPrev * self.sigmoid_derivative(store['Z' + str(l)])
            dW = dZ.dot(store['A' + str(l-1)].T) / self.n
            db = np.sum(dZ, axis = 1, keepdims = True) / self.n
            if l > 1:
                dAPrev = store['W' + str(l)].T.dot(dZ)
            derivatives["dW" + str(l)] = dW
            derivatives["db" + str(l)] = db
            dAPrev_0 = store['W' + str(l)].T.dot(dZ)

        return dAPrev_0

    def fit(self, X, Y, learning_rate=0.01, n_iterations=300):
        self.n = X.shape[0]
 
        self.layers_size.insert(0, X.shape[1])
 
        self.initialize_parameters()
        for loop in range(n_iterations):
            A, store = self.forward(X)
            cost = -np.mean(Y * np.log(A.T+ 1e-8))
            derivatives = self.backward(X, Y, store)
 
            for l in range(1, self.L + 1):
                self.parameters["W" + str(l)] = self.parameters["W" + str(l)] - learning_rate * derivatives["dW" + str(l)]
                self.parameters["b" + str(l)] = self.parameters["b" + str(l)] - learning_rate * derivatives["db" + str(l)]
 
            if loop % 100 == 0:
                print("Cost: ", cost, "Train Accuracy:", self.predict(X, Y))
 
            if loop % 10 == 0:
                self.costs.append(cost)

    def predict(self, X, Y):
        A, cache = self.forward(X)
        y_hat = np.argmax(A, axis=0)
        Y = np.argmax(Y, axis=1)
        accuracy = (y_hat == Y).mean()
        return accuracy * 100
    
    def predict_output(self, X):
        A, cache = self.forward(X)
        y_hat = np.argmax(A, axis=0)
        return y_hat

    def predict_rgb_proba(self,X):
        A, _ = self.forward(X)

        return A

    def plot_cost(self):
        plt.figure()
        plt.plot(np.arange(len(self.costs)), self.costs)
        plt.xlabel("epochs")
        plt.ylabel("cost")
        plt.show()

def pre_process_data(train_x, train_y):
    # Normalize
    train_x = train_x / 255.
 
    enc = OneHotEncoder(sparse=False, categories='auto')
    train_y = enc.fit_transform(train_y.reshape(len(train_y), -1))
    
    train_x_flatten = train_x.reshape(train_x.shape[0],-1)
    
    return train_x_flatten, train_y

In [ ]:
def rgb_predict(X_R, X_G, X_B, Y):
    A = X_R + X_G + X_B
    y_hat = np.argmax(A, axis=0)
    Y = np.argmax(Y, axis=1)
    accuracy = (y_hat == Y).mean()
    return accuracy * 100

### original 데이터로 학습한 모델

In [ ]:
data_R = data_original_train[:,:,:,0]
data_G = data_original_train[:,:,:,1]
data_B = data_original_train[:,:,:,2]

In [ ]:
if __name__ == '__main__':
    (train_x_R, train_y_R) = data_R , labels_test_original_train


    train_x_R, train_y_R = pre_process_data(train_x_R, train_y_R)

    print("train_x_R's shape: " + str(train_x_R.shape))
    print("train_y_R's shape: " + str(train_y_R.shape))
     
    layers_dims = [50, 3]
 
    ann_R = ANN(layers_dims)

    ann_R.fit(train_x_R, train_y_R, learning_rate=0.1, n_iterations=1000)

    prediction_R = ann_R.predict_rgb_proba(train_x_R)

    print(prediction_R.shape)

In [ ]:
if __name__ == '__main__':
    (train_x_G, train_y_G) = data_G , labels_test_original_train


    train_x_G, train_y_G = pre_process_data(train_x_G, train_y_G)

    print("train_x_G's shape: " + str(train_x_G.shape))
    print("train_y_G's shape: " + str(train_y_G.shape))
     
    layers_dims = [50, 3]
 
    ann_G = ANN(layers_dims)

    ann_G.fit(train_x_G, train_y_G, learning_rate=0.1, n_iterations=1000)

    prediction_G = ann_G.predict_rgb_proba(train_x_G)

    print(prediction_G.shape)

In [ ]:
if __name__ == '__main__':
    (train_x_B, train_y_B) = data_B , labels_test_original_train


    train_x_B, train_y_B = pre_process_data(train_x_B, train_y_B)

    print("train_x_B's shape: " + str(train_x_B.shape))
    print("train_y_B's shape: " + str(train_y_B.shape))
     
    layers_dims = [50, 3]
 
    ann_B = ANN(layers_dims)

    ann_B.fit(train_x_B, train_y_B, learning_rate=0.1, n_iterations=1000)

    prediction_B = ann_B.predict_rgb_proba(train_x_B)

    print(prediction_B.shape)

In [ ]:
print('rgb TRAIN Accuracy is : ', rgb_predict(prediction_R,prediction_G,prediction_B,train_y_B))

### attacked + originial 반 씩 학습한 모델

In [ ]:
if __name__ == '__main__':
    (train_x_R, train_y_R) = train_R , labels_test_original_train


    train_x_R, train_y_R = pre_process_data(train_x_R, train_y_R)

    print("train_x_R's shape: " + str(train_x_R.shape))
    print("train_y_R's shape: " + str(train_y_R.shape))
     
    layers_dims = [50, 3]
 
    ann_R_adTraining = ANN(layers_dims)

    ann_R_adTraining.fit(train_x_R, train_y_R, learning_rate=0.1, n_iterations=1000)

    prediction_R = ann_R_adTraining.predict_rgb_proba(train_x_R)

    print(prediction_R.shape)

In [ ]:
if __name__ == '__main__':
    (train_x_G, train_y_G) = train_G , labels_test_original_train


    train_x_G, train_y_G = pre_process_data(train_x_G, train_y_G)

    print("train_x_G's shape: " + str(train_x_G.shape))
    print("train_y_G's shape: " + str(train_y_G.shape))
     
    layers_dims = [50, 3]
 
    ann_G_adTraining = ANN(layers_dims)

    ann_G_adTraining.fit(train_x_G, train_y_G, learning_rate=0.1, n_iterations=1000)

    prediction_G = ann_G_adTraining.predict_rgb_proba(train_x_G)

    print(prediction_G.shape)

In [ ]:
if __name__ == '__main__':
    (train_x_B, train_y_B) = train_B , labels_test_original_train


    train_x_B, train_y_B = pre_process_data(train_x_B, train_y_B)

    print("train_x_B's shape: " + str(train_x_B.shape))
    print("train_y_B's shape: " + str(train_y_B.shape))
     
    layers_dims = [50, 3]
 
    ann_B_adTraining = ANN(layers_dims)

    ann_B_adTraining.fit(train_x_B, train_y_B, learning_rate=0.1, n_iterations=1000)

    prediction_B = ann_B_adTraining.predict_rgb_proba(train_x_B)

    print(prediction_B.shape)

In [ ]:
print('rgb TRAIN Accuracy is : ', rgb_predict(prediction_R,prediction_G,prediction_B,train_y_B))

Test 데이터 셋 만들어주고 모델의 robust 정도 측정

1. test 데이터를 모두 original로
2. test 데이터를 모두 attacked로 
3. test 데이터를 original 50% + attacked 50%로

In [ ]:
total_list = []
for i in range(len(data_attacked_test)):
    total_list.append(i)

print(total_list)
print(len(total_list))

In [ ]:
import random

attack_list = []
ran_num = random.randint(0,len(data_attacked_test))

for i in range(int(len(data_attacked_test)/2)):
    while ran_num in attack_list:
        ran_num = random.randint(0,len(data_attacked_test -1))
    attack_list.append(ran_num)

attack_list.sort()
print(attack_list)
print(len(attack_list))

In [ ]:
original_list = list(set(total_list) - set(attack_list))

print('original list : ',original_list)
print(len(original_list))
print('attack list : ',attack_list)
print(len(attack_list))

In [ ]:
# attack (50%) + original (50%)로 데이터 셋을 구성
test_data = []
for i in total_list:
    if i in attack_list:
        test_data.append(data_attacked_test[i])
    else:
        test_data.append(data_original_test[i])
test_data = np.array(test_data)

print(test_data.shape)

In [ ]:
for i in range(10):
    plt.imshow(test_data[i])
    plt.show()

In [ ]:
test_R = test_data[:,:,:,0]
test_G = test_data[:,:,:,1]
test_B = test_data[:,:,:,2]

In [ ]:
attacked_test_R = data_attacked_test[:,:,:,0]
attacked_test_G = data_attacked_test[:,:,:,1]
attacked_test_B = data_attacked_test[:,:,:,2]

original_test_R = data_original_test[:,:,:,0]
original_test_G = data_original_test[:,:,:,1]
original_test_B = data_original_test[:,:,:,2]

1. original data로만 test 진행

1-1. original model

1-2. attacked + original model

1-1. original model

In [ ]:
if __name__ == '__main__':
    (test_x_R, test_y_R) = original_test_R , labels_test_original_test

    test_x_R, test_y_R = pre_process_data(test_x_R, test_y_R)

    prediction_test_R = ann_R.predict_rgb_proba(test_x_R)

In [ ]:
if __name__ == '__main__':
    (test_x_G, test_y_G) = original_test_G , labels_test_original_test

    test_x_G, test_y_G = pre_process_data(test_x_G, test_y_G)

    prediction_test_G = ann_G.predict_rgb_proba(test_x_G)

In [ ]:
if __name__ == '__main__':
    (test_x_B, test_y_B) = original_test_B , labels_test_original_test

    test_x_B, test_y_B = pre_process_data(test_x_B, test_y_B)

    prediction_test_B = ann_B.predict_rgb_proba(test_x_B)

In [ ]:
print('rgb TEST Accuracy is : ', rgb_predict(prediction_test_R,prediction_test_G,prediction_test_B,test_y_R))

1-2. attacked + original model 

In [ ]:
if __name__ == '__main__':
    (test_x_R, test_y_R) = original_test_R , labels_test_original_test

    test_x_R, test_y_R = pre_process_data(test_x_R, test_y_R)

    prediction_test_R = ann_R_adTraining.predict_rgb_proba(test_x_R)

In [ ]:
prediction_test_R.shape

In [ ]:
if __name__ == '__main__':
    (test_x_G, test_y_G) = original_test_G , labels_test_original_test

    test_x_G, test_y_G = pre_process_data(test_x_G, test_y_G)

    prediction_test_G = ann_G_adTraining.predict_rgb_proba(test_x_G)

In [ ]:
prediction_test_G.shape

In [ ]:
if __name__ == '__main__':
    (test_x_B, test_y_B) = original_test_B , labels_test_original_test

    test_x_B, test_y_B = pre_process_data(test_x_B, test_y_B)

    prediction_test_B = ann_B_adTraining.predict_rgb_proba(test_x_B)

In [ ]:
prediction_test_B.shape

In [ ]:
print('rgb TEST Accuracy is : ', rgb_predict(prediction_test_R,prediction_test_G,prediction_test_B,test_y_R))

공격이 가해지지 않은 original data에 대해서 Adversarial Training 모델이 original 모델보다 더 부정확했다. adversarial training은 모델을 robust하게 만들지만 정확도는 낮추는 trade-off가 일어난다는 것을 확인할 수 있다.

2. attacked data로만 test 진행

2-1. orignal model

In [ ]:
if __name__ == '__main__':
    (test_x_R, test_y_R) = attacked_test_R , labels_test_original_test

    test_x_R, test_y_R = pre_process_data(test_x_R, test_y_R)

    prediction_test_R = ann_R.predict_rgb_proba(test_x_R)

In [ ]:
if __name__ == '__main__':
    (test_x_G, test_y_G) = attacked_test_G , labels_test_original_test

    test_x_G, test_y_G = pre_process_data(test_x_G, test_y_G)

    prediction_test_G = ann_G.predict_rgb_proba(test_x_G)

In [ ]:
if __name__ == '__main__':
    (test_x_B, test_y_B) = attacked_test_B , labels_test_original_test

    test_x_B, test_y_B = pre_process_data(test_x_B, test_y_B)

    prediction_test_B = ann_B.predict_rgb_proba(test_x_B)

In [ ]:
print('rgb TEST Accuracy is : ', rgb_predict(prediction_test_R,prediction_test_G,prediction_test_B,test_y_R))

2-2. attacked + original model

In [ ]:
if __name__ == '__main__':
    (test_x_R, test_y_R) = attacked_test_R , labels_test_original_test

    test_x_R, test_y_R = pre_process_data(test_x_R, test_y_R)

    prediction_test_R = ann_R_adTraining.predict_rgb_proba(test_x_R)

In [ ]:
if __name__ == '__main__':
    (test_x_G, test_y_G) = attacked_test_G , labels_test_original_test

    test_x_G, test_y_G = pre_process_data(test_x_G, test_y_G)

    prediction_test_G = ann_G_adTraining.predict_rgb_proba(test_x_G)

In [ ]:
if __name__ == '__main__':
    (test_x_B, test_y_B) = attacked_test_B , labels_test_original_test

    test_x_B, test_y_B = pre_process_data(test_x_B, test_y_B)

    prediction_test_B = ann_B_adTraining.predict_rgb_proba(test_x_B)

In [ ]:
print('rgb TEST Accuracy is : ', rgb_predict(prediction_test_R,prediction_test_G,prediction_test_B,test_y_R))

original data로 학습한 모델에 대해서는 attacked data로만 구성된 test data를 넣었을 때 22%라는 매우 저조한 인식률을 보였다. 반면에 original + attacked data로 학습한 모델은 97%라는 매우 좋은 인식률을 보였다.

다만 확인해보아야할 점은 original + attacked data로 학습한 모델에 original data를 넣었을 때는 96.9%의 정확률을, attacked data를 넣었을 때는 97.6%의 정확률을 보였다. 오히려 attacked data를 더 잘 인식했다.


--> 이는 target 값을 늘리고 target 당 data의 비중을 동일하게 만들어 다시 확인해보아야할 듯 하다.

3. original 50% + attacked 50% 합친 데이터 셋으로 학습

3-1. original model

In [ ]:
if __name__ == '__main__':
    (test_x_R, test_y_R) = test_R , labels_test_original_test

    test_x_R, test_y_R = pre_process_data(test_x_R, test_y_R)

    prediction_test_R = ann_R.predict_rgb_proba(test_x_R)

In [ ]:
if __name__ == '__main__':
    (test_x_G, test_y_G) = test_G , labels_test_original_test

    test_x_G, test_y_G = pre_process_data(test_x_G, test_y_G)

    prediction_test_G = ann_G.predict_rgb_proba(test_x_G)

In [ ]:
if __name__ == '__main__':
    (test_x_B, test_y_B) = test_B , labels_test_original_test

    test_x_B, test_y_B = pre_process_data(test_x_B, test_y_B)

    prediction_test_B = ann_B.predict_rgb_proba(test_x_B)

In [ ]:
print('rgb TEST Accuracy is : ', rgb_predict(prediction_test_R,prediction_test_G,prediction_test_B,test_y_R))

3-2. attacked + original model

In [ ]:
if __name__ == '__main__':
    (test_x_R, test_y_R) = test_R , labels_test_original_test

    test_x_R, test_y_R = pre_process_data(test_x_R, test_y_R)

    prediction_test_R = ann_R_adTraining.predict_rgb_proba(test_x_R)

In [ ]:
if __name__ == '__main__':
    (test_x_G, test_y_G) = test_G , labels_test_original_test

    test_x_G, test_y_G = pre_process_data(test_x_G, test_y_G)

    prediction_test_G = ann_G_adTraining.predict_rgb_proba(test_x_G)

In [ ]:
if __name__ == '__main__':
    (test_x_B, test_y_B) = test_B , labels_test_original_test

    test_x_B, test_y_B = pre_process_data(test_x_B, test_y_B)

    prediction_test_B = ann_B_adTraining.predict_rgb_proba(test_x_B)

In [ ]:
print('rgb TEST Accuracy is : ', rgb_predict(prediction_test_R,prediction_test_G,prediction_test_B,test_y_R))

attacked + original data에 대해서 original 모델은 60%로 예상한 것과 같이 공격 data에 대해 제대로 인식하지 못하는 모습을 보였다. attacked + original test data에 대해서는 96%의 인식률을 보여 adversarial training이 모델을 robust하게 만든다는 것을 확인하였다.

--> attacked + original train data로 만든 모델이 attacked + original < original < attacked 순의 test data 정확도를 보였다. 직관과 다르기에 확인해봐야한다.

## eps 값에 따른 adversarial training 성능 확인

이상하다 싶을 정도로 ROBUST한 모델이 만들어짐.. 이상하다

Epsilon 값을 높여 어느정도까지 방어가 되는 지 확인하자

In [ ]:
import natsort

imgs_path = "Original_test"
data_list = []
labels_list = []

for img in natsort.natsorted(os.listdir(imgs_path)):
    im = Image.open(imgs_path +'/'+ img)
    im = im.resize((32,32))
    im = np.array(im)
    data_list.append(im)
data_original_test = np.array(data_list)

labels_list_original_test = pd.read_csv('predictions_test.csv')

labels_test_original_test = np.array(labels_list_original_test['Original'])
print("Done")

In [ ]:
print(data_original_test.shape)
print(labels_test_original_test.shape)

In [ ]:
for i in range(2):
    print(labels_test_original_test[i])
    plt.imshow(data_original_test[i])
    plt.show()

In [ ]:
test_R = data_original_test[:,:,:,0]
test_G = data_original_test[:,:,:,1]
test_B = data_original_test[:,:,:,2]

In [ ]:
for i in range(2):
    plt.imshow(test_R[i])
    plt.show()

In [ ]:
if __name__ == '__main__':
    (test_x_R, test_y_R) = test_R , labels_test_original_test

    test_x_R, test_y_R = pre_process_data(test_x_R, test_y_R)

    prediction_test_R = ann_R.predict_rgb_proba(test_x_R)

In [ ]:
if __name__ == '__main__':
    (test_x_G, test_y_G) = test_G , labels_test_original_test

    test_x_G, test_y_G = pre_process_data(test_x_G, test_y_G)

    prediction_test_G = ann_G.predict_rgb_proba(test_x_G)

In [ ]:
if __name__ == '__main__':
    (test_x_B, test_y_B) = test_B , labels_test_original_test

    test_x_B, test_y_B = pre_process_data(test_x_B, test_y_B)

    prediction_test_B = ann_B.predict_rgb_proba(test_x_B)

In [ ]:
print('rgb TEST Accuracy is : ', rgb_predict(prediction_test_R,prediction_test_G,prediction_test_B,test_y_R))

In [ ]:
epsilons = [0, 0.03, .05,.06,.07,0.08, .1]

In [ ]:
# FGSM 공격 코드
def fgsm_attack(image, epsilon, data_grad):
    # data_grad 의 요소별 부호 값을 얻어옵니다
    sign_data_grad = np.sign(data_grad)
    perturbed_image = image + epsilon*sign_data_grad.T
    # 값 범위를 [0,1]로 유지하기 위해 자르기(clipping)를 추가합니다
    perturbed_image = np.clip(perturbed_image, 0, 1)
    # 작은 변화가 적용된 이미지를 리턴합니다
    return perturbed_image

In [ ]:
## predict_output에 한개씩 넣는 것부터 구현해야한다. (전체가 들어가는 것만 정상 작동함)

def test(model_R, model_G, model_B, test_x_R, test_y_R, test_x_G, test_y_G, test_x_B, test_y_B, epsilon, layers_dims):

    # 정확도 카운터
    correct = 0
    adv_examples = []
    save_original_examples = []
    save_adv_examples = []
    save_original_output = []
    save_attacked_output = []

    # 테스트 셋의 모든 예제에 대해 루프를 돕니다
    
    for i in range(len(test_x_R)):
        data_R = test_x_R[i].reshape(1,1024)
        target_R = test_y_R[i].reshape(1,3)
        data_G = test_x_G[i].reshape(1,1024)
        target_G = test_y_G[i].reshape(1,3)
        data_B = test_x_B[i].reshape(1,1024)
        target_B = test_y_B[i].reshape(1,3)

        A_R,store_R = model_R.forward(data_R)
        A_G,store_G = model_G.forward(data_G)
        A_B,store_B = model_B.forward(data_B)

        predict_R = model_R.predict_rgb_proba(data_R)
        predict_G = model_G.predict_rgb_proba(data_G)
        predict_B = model_B.predict_rgb_proba(data_B)


        prediction = predict_R + predict_G + predict_B
        output = np.argmax(prediction, axis=0)

        init_output = int(output)
        
        if init_output != int(np.argmax(target_R,axis = 1)):
            continue


        data_grad_R = model_R.backward_ad(data_R,target_R,store_R)
        
        perturbed_data_R = fgsm_attack(data_R, epsilon, data_grad_R)

        data_grad_G = model_G.backward_ad(data_G,target_G,store_G)
        
        perturbed_data_G = fgsm_attack(data_G, epsilon, data_grad_G)

        data_grad_B = model_B.backward_ad(data_B,target_B,store_B)
        
        perturbed_data_B = fgsm_attack(data_B, epsilon, data_grad_B)


        output_R = model_R.predict_rgb_proba(perturbed_data_R)
        output_G = model_G.predict_rgb_proba(perturbed_data_G)
        output_B = model_B.predict_rgb_proba(perturbed_data_B)

        prediction_rgb = output_R + output_G + output_B
        output = np.argmax(prediction_rgb, axis=0)

        final_pred = int(output)

        if final_pred == int(np.argmax(target_R,axis = 1)):
            correct += 1
            if (epsilon == 0) and (len(adv_examples) < 5):
                perturbed_data = np.stack((perturbed_data_R, perturbed_data_G, perturbed_data_B),axis=2)
                adv_ex = perturbed_data
                adv_examples.append((init_output,final_pred,perturbed_data))
        else:
            if len(adv_examples) < 5:
                perturbed_data = np.stack((perturbed_data_R, perturbed_data_G, perturbed_data_B),axis=2)
                adv_ex = perturbed_data
                adv_examples.append((init_output,final_pred,perturbed_data))

        save_original_examples.append(np.stack((data_R, data_G, data_B),axis=2))
        save_adv_examples.append(np.stack((perturbed_data_R, perturbed_data_G, perturbed_data_B),axis=2))
        save_original_output.append(init_output)
        save_attacked_output.append(final_pred)

    # 해당 엡실론에서의 최종 정확도를 계산합니다
    final_acc = correct/float(len(test_x_R))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(test_x_R), final_acc))

    # 정확도와 적대적 예제를 리턴합니다
    return final_acc, adv_examples, save_original_examples, save_adv_examples, save_original_output, save_attacked_output

In [ ]:
accuracies = []
examples = []

# 각 엡실론에 대해 테스트 함수를 실행합니다
for eps in epsilons:
    acc, ex, _, _, _, _ = test(ann_R, ann_G, ann_B, test_x_R, test_y_R, test_x_G, test_y_G, test_x_B, test_y_B, eps, layers_dims)
    accuracies.append(acc)
    examples.append(ex)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons, accuracies, "*-")
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .12, step=0.02))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
# 각 엡실론에서 적대적 샘플의 몇 가지 예를 도식화합니다
cnt = 0
plt.figure(figsize=(8,10))
for i in range(len(epsilons)):
    for j in range(len(examples[i])):
        cnt += 1
        plt.subplot(len(epsilons),len(examples[0]),cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
        orig,adv,ex = examples[i][j]
        plt.title("{} -> {}".format(orig, adv))
        plt.imshow(ex.reshape((32,32,3)), cmap="gray")
plt.tight_layout()
plt.show()

In [ ]:
_,_, save_original_examples, saved_adversarial_examples, saved_original_output, saved_attacked_output = test(ann_R, ann_G, ann_B, test_x_R, test_y_R, test_x_G, test_y_G, test_x_B, test_y_B, 0.06, layers_dims)

In [ ]:
saved_adversarial_examples = np.array(saved_adversarial_examples)
saved_original_output = np.array(saved_original_output)

In [ ]:
saved_adversarial_examples.shape

In [ ]:
saved_adversarial_examples = saved_adversarial_examples.reshape(-1,32,32,3)

In [ ]:
saved_original_output.shape

In [ ]:
print(saved_original_output[1])
plt.imshow(saved_adversarial_examples[1])

In [ ]:
test_R = saved_adversarial_examples[:,:,:,0]
test_G = saved_adversarial_examples[:,:,:,1]
test_B = saved_adversarial_examples[:,:,:,2]

In [ ]:
test_R.shape

In [ ]:
if __name__ == '__main__':
    (test_x_R, test_y_R) = test_R , saved_original_output

    test_x_R, test_y_R = pre_process_data(test_x_R, test_y_R)

    prediction_test_R = ann_R.predict_rgb_proba(test_x_R)

In [ ]:
if __name__ == '__main__':
    (test_x_G, test_y_G) = test_G , saved_original_output

    test_x_G, test_y_G = pre_process_data(test_x_G, test_y_G)

    prediction_test_G = ann_G.predict_rgb_proba(test_x_G)

In [ ]:
if __name__ == '__main__':
    (test_x_B, test_y_B) = test_B , saved_original_output

    test_x_B, test_y_B = pre_process_data(test_x_B, test_y_B)

    prediction_test_B = ann_B.predict_rgb_proba(test_x_B)

In [ ]:
print('rgb TEST Accuracy is : ', rgb_predict(prediction_test_R,prediction_test_G,prediction_test_B,test_y_R))

??? 왜 test 함수에서의 정확도와 rgb_predict 함수에서의 정확도가 다를까 ....?????